<a href="https://colab.research.google.com/github/eralpucar/PIWORKASSESMENTS/blob/main/piworksassesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# QUESTION 4 and 5 they are same
import pandas as pd
df=pd.read_csv("country_vaccination_stats.csv")
result = pd.DataFrame(columns=df.columns)
for country in df["country"].unique():
    subset = df[df["country"] == country].copy()
    if len(subset) > 1:
        subset["daily_vaccinations"].fillna(subset["daily_vaccinations"].min(), inplace=True)
    else:
        subset["daily_vaccinations"].fillna(0, inplace=True)
    result = pd.concat([result, subset])

result.reset_index(drop=True, inplace=True)
result.isna().sum()

country               0
date                  0
daily_vaccinations    0
vaccines              0
dtype: int64

In [54]:
# QUESTION 6
result.groupby("country").agg({"daily_vaccinations":"median"}).sort_values(by="daily_vaccinations",ascending=False).head(3)

,daily_vaccinations
country,
United States,378253.0
China,276786.0
India,173922.0


In [47]:
#QUESTION 7
result[result["date"]=="1/6/2021"]["daily_vaccinations"].sum()


1485255.0

In [52]:
#QUESTION 8
query = """
UPDATE df
SET daily_vaccinations =
    (SELECT
        CASE
            WHEN daily_vaccinations IS NULL THEN
                (SELECT MEDIAN(daily_vaccinations)
                 FROM df AS t
                 WHERE df.country = t.country AND daily_vaccinations IS NOT NULL)
            ELSE daily_vaccinations
        END
    )
WHERE EXISTS
    (SELECT 1
     FROM df AS t
     WHERE df.country = t.country);
"""